In [6]:
import pandas as pd

In [7]:
df = pd.read_json('/Users/ayushpatne/Documents/Flutter/expense_tracker/lib/data/transaction2.json')
df

,title,amount,date,category
0,Transaction1 on Fitness,53,2023-03-01,Fitness
1,Transaction2 on Entertainment,168,2023-03-01,Entertainment
2,Transaction3 on Food,139,2023-03-01,Food
3,Transaction4 on Transportation,156,2023-03-01,Transportation
4,Transaction5 on Food,65,2023-03-01,Food
...,...,...,...,...
7295,Transaction16 on Food,67,2024-02-28,Food
7296,Transaction17 on Shopping,55,2024-02-28,Shopping
7297,Transaction18 on Fitness,84,2024-02-28,Fitness
7298,Transaction19 on Fitness,143,2024-02-28,Fitness


In [8]:
monthwise_data = {

}
monthly_categorywise_data = {

}

for index, row in df.iterrows():
    date_obj = pd.to_datetime(row['date'])
    month = date_obj.month
    year = date_obj.year

    category = row['category']

    if f'{month}, {year}' not in monthwise_data :
        monthwise_data[f'{month}, {year}'] = 0
        monthwise_data[f'{month}, {year}'] += row['amount']

    else : 
        monthwise_data[f'{month}, {year}'] += row['amount']
        # monthly_categorywise_data[f'{month}, {year}'][category] += row['amount']
        
        
    if  f'{month}, {year}' not in monthly_categorywise_data: 
        monthly_categorywise_data[f'{month}, {year}']  = {}
        if category not in  monthly_categorywise_data[f'{month}, {year}']:
            monthly_categorywise_data[f'{month}, {year}'][category]  = 0
            monthly_categorywise_data[f'{month}, {year}'][category] += row['amount']
        
    else:
        if category not in  monthly_categorywise_data[f'{month}, {year}']:
            monthly_categorywise_data[f'{month}, {year}'][category]  = 0
            monthly_categorywise_data[f'{month}, {year}'][category] += row['amount']
        else:
            monthly_categorywise_data[f'{month}, {year}'][category] += row['amount']
                

       

    

# monthwise_data
monthwise_data
    

{'3, 2023': 64126,
 '4, 2023': 62802,
 '5, 2023': 63854,
 '6, 2023': 61705,
 '7, 2023': 64916,
 '8, 2023': 63391,
 '9, 2023': 61287,
 '10, 2023': 60936,
 '11, 2023': 60334,
 '12, 2023': 66059,
 '1, 2024': 60796,
 '2, 2024': 56433}

In [9]:
#Making the monthlywise_data usable for regression
months_list_from_monthwise_data = list(keys.split(',')[0] for keys in monthwise_data.keys())
amount_list_from_monthwise_data = list(monthwise_data.values())

months_list_from_monthwise_data

['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '1', '2']

# Regression

In [10]:

monthly_expenses = pd.DataFrame([months_list_from_monthwise_data, amount_list_from_monthwise_data ],  ).T.rename(columns={0: 'Months', 1: 'Amount'})
monthly_expenses

,Months,Amount
0,3,64126
1,4,62802
2,5,63854
3,6,61705
4,7,64916
5,8,63391
6,9,61287
7,10,60936
8,11,60334
9,12,66059


In [11]:
from sklearn.model_selection import train_test_split

X = monthly_expenses['Months'].values.reshape(-1, 1)  # Feature: Month
y = monthly_expenses['Amount'].values  # Target: Monthly Expense

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [12]:
from sklearn.linear_model import LinearRegression
import pickle
# Create and train a Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# with open('regression_model.pkl', 'wb') as file:
#     pickle.dump(model, file)

LinearRegression()

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predict on the testing set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)

Mean Absolute Error: 2188.469892473118
Root Mean Squared Error: 2656.464209929916


In [14]:
next_months = [4, 5, 6]
future_expenses = model.predict([[m] for m in next_months])
print("Predicted Expenses for Next 3 Months:", future_expenses)

Predicted Expenses for Next 3 Months: [61242.22258065 61414.45322581 61586.68387097]


# Category Of Expense groupby()

In [16]:
df

,title,amount,date,category
0,Transaction1 on Fitness,53,2023-03-01,Fitness
1,Transaction2 on Entertainment,168,2023-03-01,Entertainment
2,Transaction3 on Food,139,2023-03-01,Food
3,Transaction4 on Transportation,156,2023-03-01,Transportation
4,Transaction5 on Food,65,2023-03-01,Food
...,...,...,...,...
7295,Transaction16 on Food,67,2024-02-28,Food
7296,Transaction17 on Shopping,55,2024-02-28,Shopping
7297,Transaction18 on Fitness,84,2024-02-28,Fitness
7298,Transaction19 on Fitness,143,2024-02-28,Fitness


In [31]:
months_col = []
for index, row in df.iterrows():
    date_obj = pd.to_datetime(row['date'])
    month = date_obj.month_name()
    year = date_obj.year

    months_col.append(f'{month}, {year}')

df['month'] = months_col


In [32]:
df

,title,amount,date,category,month
0,Transaction1 on Fitness,53,2023-03-01,Fitness,"March, 2023"
1,Transaction2 on Entertainment,168,2023-03-01,Entertainment,"March, 2023"
2,Transaction3 on Food,139,2023-03-01,Food,"March, 2023"
3,Transaction4 on Transportation,156,2023-03-01,Transportation,"March, 2023"
4,Transaction5 on Food,65,2023-03-01,Food,"March, 2023"
...,...,...,...,...,...
7295,Transaction16 on Food,67,2024-02-28,Food,"February, 2024"
7296,Transaction17 on Shopping,55,2024-02-28,Shopping,"February, 2024"
7297,Transaction18 on Fitness,84,2024-02-28,Fitness,"February, 2024"
7298,Transaction19 on Fitness,143,2024-02-28,Fitness,"February, 2024"


In [43]:
groupby_category_df = df.drop(columns=['date']).groupby(['month', 'category'])
groupby_category_df.sum().drop(columns=['title'])

amount
month           category              
April, 2023     Entertainment    13444
                Fitness          12977
                Food             11419
                Shopping         13602
                Transportation   11360
August, 2023    Entertainment    11490
                Fitness          13454
                Food             12258
                Shopping         13390
                Transportation   12799
December, 2023  Entertainment    12977
                Fitness          11651
                Food             13276
                Shopping         12258
                Transportation   15897
February, 2024  Entertainment    12309
                Fitness          10233
                Food             11569
                Shopping         12090
                Transportation   10232
January, 2024   Entertainment    13754
                Fitness          10361
                Food             13851
                Shopping         10735
                Transportation   12095
July, 2023      Entertainment    12877
                Fitness          13188
                Food             13604
                Shopping         11682
                Transportation   13565
June, 2023      Entertainment    12749
                Fitness          13019
                Food             10416
                Shopping         13964
                Transportation   11557
March, 2023     Entertainment    12221
                Fitness          11396
                Food             13773
                Shopping         13455
                Transportation   13281
May, 2023       Entertainment    12529
                Fitness          13447
                Food             13860
                Shopping         11208
                Transportation   12810
November, 2023  Entertainment    13196
                Fitness          12491
                Food             10318
                Shopping         11075
                Transportation   13254
October, 2023   Entertainment    11822
                Fitness          12932
                Food             12259
                Shopping         12303
                Transportation   11620
September, 2023 Entertainment    11998
                Fitness          13049
                Food             12068
                Shopping         12030
                Transportation   12142

In [48]:
ls = []
for category in ["Food", "Entertainment", "Transportation", "Shopping", "Fitness"]:
    amt = groupby_category_df.get_group(('March, 2023', category)).sum()['amount']
    ls.append([category, amt])


ls

[['Food', 13773],
 ['Entertainment', 12221],
 ['Transportation', 13281],
 ['Shopping', 13455],
 ['Fitness', 11396]]

In [56]:
import pandas as pd
df = pd.read_json('/Users/ayushpatne/Documents/Flutter/expense_tracker/lib/data/transaction2.json')


def modify_dataset():
    months_col = []
    for index, row in df.iterrows():
        date_obj = pd.to_datetime(row['date'])
        month = date_obj.month_name()
        year = date_obj.year

        months_col.append(f'{month}, {year}')

    df['month'] = months_col

    

def get_category_and_max_category():
    modify_dataset()
    groupby_category_df = df.drop(columns=['date']).groupby(['month', 'category'])
    groupby_category_df.sum().drop(columns=['title'])

    ls = []
    for category in ["Food", "Entertainment", "Transportation", "Shopping", "Fitness"]:
        amt = groupby_category_df.get_group(('March, 2023', category)).sum()['amount']
        ls.append([category, amt])
    
    return ls, max(ls)

print(get_category_and_max_category())

([['Food', 13773], ['Entertainment', 12221], ['Transportation', 13281], ['Shopping', 13455], ['Fitness', 11396]], ['Transportation', 13281])


In [54]:
from datetime import datetime as dt
import datetime
def category_most_spent():

    current = dt.now().month - 1
    name = datetime.date(1900, current, 1).strftime('%B')

    current_year = dt.now().year

    # summary, max_spent_category = get_category_summary.get_category_and_max_category(
    #     f"{name}, {current_year}"
    # )
    return f"{name}, and max : {current_year}"

category_most_spent()

'March, and max : 2024'